# Utilizando o Cassandra

Neste notebook, há um node de Apache Cassandra executando. O campo abaixo conecta ao DBMS e cria o Keyspace na qual o schema será criado e dados fictícios serão inseridos.

In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'], port = 9042)
session = cluster.connect()
session.execute('CREATE KEYSPACE IF NOT EXISTS brecho WITH REPLICATION = { \'class\' : \'SimpleStrategy\', \'replication_factor\' : \'1\' };')
session.execute('USE brecho;')

## Criação do schema

Para a execução das consultas, duas tabelas serão criadas:

In [ ]:
session.execute('CREATE TABLE IF NOT EXISTS feedback_por_produto(produto_id uuid, pedido_id uuid, data timestamp, nota double, comentarios text, PRIMARY KEY ((produto_id), pedido_id));')
session.execute('CREATE TABLE IF NOT EXISTS feedback_por_vendedor(vendedor_id uuid, pedido_id uuid, data timestamp, nota double, comentarios text, PRIMARY KEY ((vendedor_id), pedido_id));')

## Inserção de dados

Com base nas tabelas criadas, uma quantidade de informação será gerada de forma a testar as consultas

In [ ]:
from datetime import datetime
import random
import uuid

random.seed("INF-325") #inicializa o seed para tornar os dados determinísticos

idProdutos = []
idVendedores = []

for p in range(50): #criando 50 produtos
    idProdutos.append(str(uuid.uuid4()))
    
for v in range(20): #criando 20 vendedores
    idVendedores.append(str(uuid.uuid4()))

#inclui registros de feedbacks de produtos
now = datetime.now()
totalRegistros = 0

#dentre os 50 produtos, cria uma quantia entre 100 e 500 avaliações por produto
print('Incluindo feedbacks de produtos')
insertQuery = 'INSERT INTO feedback_por_produto (produto_id, pedido_id, data, nota, comentarios) VALUES ({produto}, {pedido}, \'{data}\', {nota}, \'{comentarios}\');'
for ip in idProdutos:
    quantidade = random.randrange(100, 500)
    totalRegistros += quantidade
    print('Incluindo {} registros para o produto {}'.format(quantidade, ip))
          
    for x in range(quantidade):
        session.execute(insertQuery.format(produto = ip, pedido = str(uuid.uuid4()), data = datetime.utcnow(), nota = 5, comentarios = 'comentario'))

print('{} registros incluídos em {}s\n'.format(totalRegistros, (datetime.now() - now).total_seconds()))

#inclui registros de feedbacks de vendedores
now = datetime.now()
totalRegistros = 0

#dentre os 20 vendedores, cria uma quantia entre 100 e 500 avaliações por vendedor
print('Incluindo feedbacks de vendedores')
insertQuery = 'INSERT INTO feedback_por_vendedor (vendedor_id, pedido_id, data, nota, comentarios) VALUES ({produto}, {pedido}, \'{data}\', {nota}, \'{comentarios}\');'
for ip in idProdutos:
    quantidade = random.randrange(100, 500)
    totalRegistros += quantidade
    print('Incluindo {} registros para o vendedor {}'.format(quantidade, ip))
          
    for x in range(quantidade):
        session.execute(insertQuery.format(produto = ip, pedido = str(uuid.uuid4()), data = datetime.utcnow(), nota = 5, comentarios = 'comentario'))

print('{} registros incluídos em {}s'.format(totalRegistros, (datetime.now() - now).total_seconds()))

## Utilização das Consultas